In [ ]:
#pip install pyreadr

In [35]:
import pandas as pd
import pyreadr

# Data Cleaning

## 0. Delineation File

In [75]:
delineation_df = pd.read_excel('./data/delineation_file.xls')
delineation_df = delineation_df.iloc[1:1918]
delineation_df = delineation_df.rename(columns=delineation_df.iloc[0])
delineation_df = delineation_df.drop(1)
delineation_df['FIPS State Code'] = delineation_df['FIPS State Code'].astype(int)
delineation_df

,CBSA Code,Metropolitan Division Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,Metropolitan Division Title,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,Central/Outlying County
2,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Brown County,South Dakota,46,013,Central
3,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Edmunds County,South Dakota,46,045,Outlying
4,10140,NaN,NaN,"Aberdeen, WA",Micropolitan Statistical Area,NaN,NaN,Grays Harbor County,Washington,53,027,Central
5,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,NaN,NaN,Callahan County,Texas,48,059,Outlying
6,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,NaN,NaN,Jones County,Texas,48,253,Outlying
...,...,...,...,...,...,...,...,...,...,...,...,...
1913,49700,NaN,472,"Yuba City, CA",Metropolitan Statistical Area,NaN,"Sacramento-Roseville, CA",Sutter County,California,6,101,Central
1914,49700,NaN,472,"Yuba City, CA",Metropolitan Statistical Area,NaN,"Sacramento-Roseville, CA",Yuba County,California,6,115,Central
1915,49740,NaN,NaN,"Yuma, AZ",Metropolitan Statistical Area,NaN,NaN,Yuma County,Arizona,4,027,Central
1916,49780,NaN,198,"Zanesville, OH",Micropolitan Statistical Area,NaN,"Columbus-Marion-Zanesville, OH",Muskingum County,Ohio,39,119,Central


In [128]:
code = pyreadr.read_r('./data/code.Rds')

In [65]:
code_df = code[None]
code_df['cityID'] = code_df['cityID'].astype(int)
code_df

,cityID,NAME,countyID,cbsa,ORI
0,100124,ABBEVILLE POLICE DEPARTMENT,1067,20020,AL03701
1,100460,ADAMSVILLE POLICE DEPARTMENT,1073,13820,AL00122
2,100484,ADDISON POLICE DEPARTMENT,1133,NaN,AL06703
3,1005690,BETHANY BEACH POLICE DEPARTMENT,10005,42580,DE00310
4,1005820,BETHEL PD,10005,42580,DE00331
...,...,...,...,...,...
22721,999999,WILLIMANTIC POLICE DEPARTMENT,9015,25540,CT00163
22722,999999,WILLIMANTIC POLICE DEPARTMENT,9015,35980,CT00163
22723,999999,WILLIMANTIC POLICE DEPARTMENT,9015,48740,CT00163
22724,999999,WILLIMANTIC POLICE DEPARTMENT,9015,14860,CT00163


## 1. Crime Data

In [2]:
crime_df = pd.read_csv('./data/crime.csv')

In [127]:
crime_df = crime_df.drop(columns=['ORI', 'GROUP', 'countyID', 'ORI', 'FIPS State Code'])

,Unnamed: 0,STATE,YEAR,cityID,ORI,GROUP,DIV,MSA,COUNTY,SEQNO,...,GI.empl,GI.health,GI.income,gini,eduCollege,white,medianAge,medianIncome,alcohol,cbsaTweets
0,1,alabama,2015,107912.0,AL05004,6,(6) East South Central,NaN,48,5826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,alabama,2015,107912.0,AL05004,6,(6) East South Central,NaN,48,5826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,alabama,2015,102956.0,AL04401,4,(6) East South Central,368.0,42,2262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,alabama,2015,104660.0,AL00501,6,(6) East South Central,201.0,2,3655,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,alabama,2015,162328.0,AL00401,4,(6) East South Central,556.0,1,50634,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129365,129366,ohio,2014,3999999.0,OH07625,6,(3) East North Central,128.0,76,62844,...,0.524104,0.479587,0.537649,0.4395,0.437163,0.92286,40.6,21732.0,8.0,0.0
129366,129367,ohio,2014,3988084.0,OH06001,4,(3) East North Central,NaN,60,69408,...,0.524104,0.479587,0.537649,0.4395,0.437163,0.92286,40.6,21732.0,8.0,0.0
129367,129368,ohio,2014,3988084.0,OH06001,4,(3) East North Central,NaN,60,69408,...,0.524104,0.479587,0.537649,0.4395,0.437163,0.92286,40.6,21732.0,8.0,0.0
129368,129369,ohio,2014,3988084.0,OH06001,4,(3) East North Central,NaN,60,69408,...,0.524104,0.479587,0.537649,0.4395,0.437163,0.92286,40.6,21732.0,8.0,0.0


## 2. Alcohol Data

In [121]:
alcohol_df = pd.read_csv('./data/alcohol.csv')
alcohol_df['alcohol_per_month'] = round(alcohol_df['alcohol']/12, 2)
alcohol_df = alcohol_df.fillna(0)
alcohol_df

,cbsa,YEAR,alcohol,alcohol_per_month
0,10100,2012,3.0,0.25
1,10140,2012,7.0,0.58
2,10180,2012,14.0,1.17
3,10220,2012,4.0,0.33
4,10300,2012,8.0,0.67
...,...,...,...,...
4660,49660,2016,51.0,4.25
4661,49700,2016,11.0,0.92
4662,49740,2016,8.0,0.67
4663,49780,2016,8.0,0.67


## 3. Twitter Data

In [107]:
twitter_df = pd.read_csv('./data/twitter_example.csv', encoding = "ISO-8859-1")
# only select useful columns
twitter_df = twitter_df[['id_str', 'UScounty', 'UScity', 'USstate', 'day', 'month', 'YEAR', 'user_id_str', 'lang',
                         'retweet_count', 'statuses_count', 'followers_count', 'favourites_count', 'friends_count']]
# drop missing value
twitter_df = twitter_df.dropna(subset = ['UScity', 'USstate'], how='all')
# add cbsa code
twitter_df = twitter_df.fillna(0)
twitter_df['UScity'] = twitter_df['UScity'].astype(int)
twitter_df['USstate'] = twitter_df['USstate'].astype(int)
twitter_df = pd.merge(twitter_df, code_df, left_on='UScity', right_on='cityID', how='left') #4948
twitter_df = twitter_df.drop_duplicates(subset=['id_str'])

delineation_df_small = delineation_df[['CBSA Code', 'FIPS State Code']]
twitter_df = pd.merge(twitter_df, delineation_df_small, left_on='USstate', right_on='FIPS State Code', how='left')
twitter_df = twitter_df.drop_duplicates(subset=['id_str'])
twitter_df = twitter_df.drop(columns=['cityID', 'NAME', 'countyID', 'ORI', 'FIPS State Code'])
twitter_df = twitter_df.fillna(0)
for index, row in twitter_df.iterrows():
    if twitter_df['cbsa'][index]==0:
        code = twitter_df['CBSA Code'][index]
        twitter_df['cbsa'][index] = code
twitter_df = twitter_df.drop(columns=['lang','CBSA Code', 'UScounty', 'UScity', 'USstate', 'day', 'user_id_str', 'id_str'])
twitter_df = twitter_df[twitter_df['cbsa']!=0]
twitter_df #4788

/var/folders/3h/x2xrdjwn22bbdw5d98hd13w80000gn/T/ipykernel_44595/2537842365.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter_df['cbsa'][index] = code


,month,YEAR,retweet_count,statuses_count,followers_count,favourites_count,friends_count,cbsa
0,1,2013,0,1609,228,847,222,33340
1,1,2013,0,33,0,0,0,14460
14,1,2013,1,39932,1015,16,1032,45060
15,1,2013,0,2743,108,276,98,11580
66,1,2013,2,13105,462,340,70,13720
...,...,...,...,...,...,...,...,...
118178,7,2015,0,1304,2083,0,2263,27260
118179,7,2015,0,14286,255,4243,220,10140
118207,7,2015,65,101939,1706,2350,1697,12580
118208,7,2015,53,5548,392,1281,1007,34980


In [125]:
# group by month, year, cbsa
twitter_agg1 = twitter_df.groupby(['month','YEAR','cbsa']).size().reset_index(name='tweets_counts')
twitter_agg2 = twitter_df.groupby(['month','YEAR','cbsa']).sum()
twitter_agg = pd.merge(twitter_agg1, twitter_agg2, on=['month','YEAR','cbsa'])
twitter_agg

,month,YEAR,cbsa,tweets_counts,retweet_count,statuses_count,followers_count,favourites_count,friends_count
0,1,2013,10180,10,497,69682,4528,7308,4464
1,1,2013,10420,5,23,31909,1292,4675,1414
2,1,2013,10500,1,0,16737,392,179,256
3,1,2013,10580,8,4,69768,1505,3446,1485
4,1,2013,10620,1,387,407,105,69,129
...,...,...,...,...,...,...,...,...,...
2536,12,2015,45060,1,3,4136,337,3210,601
2537,12,2015,45300,1,918,2344,285,574,226
2538,12,2015,48620,1,0,5192,443,1691,424
2539,12,2015,49420,1,3876,14266,909,14821,461


# Merge All Data

In [126]:
# alcohol and twitter
al_twitter = pd.merge(twitter_agg, alcohol_df, on=['cbsa','YEAR'], how='left')
al_twitter = al_twitter.drop(columns=['alcohol'])
al_twitter

,month,YEAR,cbsa,tweets_counts,retweet_count,statuses_count,followers_count,favourites_count,friends_count,alcohol_per_month
0,1,2013,10180,10,497,69682,4528,7308,4464,1.25
1,1,2013,10420,5,23,31909,1292,4675,1414,4.67
2,1,2013,10500,1,0,16737,392,179,256,1.58
3,1,2013,10580,8,4,69768,1505,3446,1485,10.83
4,1,2013,10620,1,387,407,105,69,129,0.17
...,...,...,...,...,...,...,...,...,...,...
2536,12,2015,45060,1,3,4136,337,3210,601,5.67
2537,12,2015,45300,1,918,2344,285,574,226,17.17
2538,12,2015,48620,1,0,5192,443,1691,424,9.83
2539,12,2015,49420,1,3876,14266,909,14821,461,0.33
